In [11]:
import pandas as pd

In [12]:
data = pd.read_csv('out.csv')

In [13]:
def read_cluster(df, i):
    col_list = df[i].tolist()
    col_set = set()
    for phrase in col_list:
        if type(phrase) == str:
            col_set.add(phrase)
    return col_set

In [14]:
all_clusters = dict()
for col in data.columns.tolist():
    boom = read_cluster(data, col)
    all_clusters[int(col)] = list(boom)


In [15]:
all_clusters.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])